# Matcha Helper - Reverse Input
### 2026 Made with ❤️, Gilang Wahyu Prasetyo
kemaren pake matcha helper untuk mencari IDSBR dan meng-GC di matchapro, kan?
kalo sekarang, gunakan ini untuk mengaktifkan filter "belum GC" di matchapro, lalu mulai
meng-GC SBR-SBR tersebut. goodluck!

# IMPORT

In [1]:
import os
import re
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException, ElementNotInteractableException
pd.set_option("display.max_columns", None)



# BUKA CHROME n REDIRECT

In [2]:
url = "https://matchapro.web.bps.go.id/dirgc"

chrome_cust = "isi di sini path nya"

user_agent = (
    "Mozilla/5.0 (Linux; Android 15; SM-G991B Build/TP1A.220624.014; wv) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 "
    "Chrome/132.0.6834.88 Mobile Safari/537.36"
)

options = Options()
options.add_argument("--start-maximized")
options.add_argument(f"--user-agent={user_agent}")

try:
    service = Service(executable_path=chrome_cust)
    driver = webdriver.Chrome(service=service, options=options)
except Exception:
    service = Service()
    driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

# Login SSO BPS

In [ ]:
username_sso = "sso"
password_sso = "pass"

wait = WebDriverWait(driver, 20)

# Klik tombol SSO
wait.until(EC.element_to_be_clickable((By.ID, "login-sso"))).click()

# Isi kredensial
wait.until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username_sso)
driver.find_element(By.ID, "password").send_keys(password_sso)

# Submit login
driver.find_element(By.ID, "kc-login").click()

# Import Spreadsheet

## sbr yang belom clean

In [21]:
sbr = pd.read_excel(r"C:\Users\ACER\Downloads\sbr.xlsx")
sbrdup = sbr.copy()

sbrdup["idsbr"] = (
    sbrdup["idsbr"]
    .astype(str)
    .str.replace(r"\.0$", "", regex=True)
    .replace("nan", pd.NA)
    .astype("Int64")  # integer nullable
)

print("jumlah baris:", len(sbrdup))

jumlah baris: 42441


## sbr yang udah clean

In [ ]:
# Google Sheets ID dan list GID
sheet_id = "isi_link_spreadsheetmu_sendiri"
gids = [1251581783, 1958661473, 501907667, 1654191200, 1252202213, 0, 455439078]

# List untuk menyimpan dataframe
dfs = []

# Loop untuk import setiap GID
for gid in gids:
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"
    df = pd.read_csv(url)
    dfs.append(df)
    print(f"GID {gid}: {len(df)} baris")

# Gabungkan semua dataframe
sbrclean = pd.concat(dfs, ignore_index=True)
print(f"\nTotal setelah digabung: {len(sbrclean)} baris")
sbrclean["idsbr"] = (
    sbrclean["idsbr"]
    .astype(str)
    .str.replace(r"\.0$", "", regex=True)
    .replace("nan", pd.NA)
    .astype("Int64")  # integer nullable
)

GID 1251581783: 1528 baris
GID 1958661473: 3453 baris
GID 501907667: 1893 baris
GID 1654191200: 10846 baris
GID 1252202213: 993 baris
GID 0: 26166 baris
GID 455439078: 3306 baris

Total setelah digabung: 48185 baris


In [19]:
# Drop duplicate dari sbrclean
#sbrclean = sbrclean.drop_duplicates(subset=['idsbr', 'nama_usaha', 'alamat'], keep='first')
sbrclean = sbrclean.drop(columns=['nomor', 'Petugas'])
sbrclean = sbrclean.drop_duplicates(keep='first')

print(f"Setelah drop duplicate: {len(sbrclean)} baris")
sbrclean = sbrclean.sort_values(by='idsbr')
sbrclean.to_excel(r'C:\Users\acer\Downloads\sbrclean.xlsx', index=False)


Setelah drop duplicate: 47988 baris


# Fungsi

## fungsi tandai

In [107]:
def fungsi_tandai(df):
    """
    Fungsi untuk menandai usaha dengan data dari dataframe
    df: dataframe yang berisi kolom idsbr, keberadaan_usaha, latitude, longitude
    """
    # Get IDSBR dari detail yang sudah expand
    detail_rows = driver.find_elements(By.CSS_SELECTOR, "div.detail-row")
    idsbr = ""
    for row in detail_rows:
        label = row.find_element(By.CSS_SELECTOR, "span.detail-label").text.strip()
        if label == "#IDSBR":
            idsbr = row.find_element(By.CSS_SELECTOR, "span.detail-value").text.strip()
            print(f"IDSBR: {idsbr}")
            break
    
    # Cari data di dataframe berdasarkan idsbr
    if idsbr:
        match = df[df['idsbr'].astype(str).str.strip() == idsbr]
        if not match.empty:
            keberadaan_usaha = str(match['keberadaan_usaha'].iloc[0]).strip()
            latitude = str(match['latitude'].iloc[0]).strip()
            longitude = str(match['longitude'].iloc[0]).strip()
            print(f"Data ditemukan - Keberadaan: {keberadaan_usaha}, Lat: {latitude}, Lon: {longitude}")
        else:
            # hampir tidak mungkin sih ini
            print(f"IDSBR {idsbr} tidak ditemukan di dataframe")
            return False
    else:
        print("IDSBR kosong, mencari berdasarkan nama usaha...")
        
        # Ambil nama usaha dari card header
        try:
            card_header = driver.find_element(By.CSS_SELECTOR, "div.usaha-card-header")
            nama_usaha = card_header.find_element(By.CSS_SELECTOR, "h5.usaha-card-title").text.strip()
            print(f"Nama Usaha: {nama_usaha}")
            
            # Cari di dataframe berdasarkan nama usaha
            match = df[df['nama_usaha'].astype(str).str.strip().str.upper() == nama_usaha.upper()]
            
            if not match.empty:
                keberadaan_usaha = str(match['keberadaan_usaha'].iloc[0]).strip()
                latitude = str(match['latitude'].iloc[0]).strip()
                longitude = str(match['longitude'].iloc[0]).strip()
                print(f"Data ditemukan dari nama usaha - Keberadaan: {keberadaan_usaha}, Lat: {latitude}, Lon: {longitude}")
            else:
                print(f"Nama usaha '{nama_usaha}' tidak ditemukan di dataframe")
                return False
        except Exception as e:
            print(f"Gagal mencari nama usaha: {str(e)}")
            return False

    # Klik tombol Tandai dengan retry
    max_retry = 5
    tandai_clicked = False
    for attempt in range(max_retry):
        try:
            tandai_btn = driver.find_element(By.CSS_SELECTOR, "button.btn-tandai")
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", tandai_btn)
            time.sleep(0.3)
            tandai_btn.click()
            print("Tombol Tandai berhasil diklik")
            tandai_clicked = True
            break
        except:
            print(f"Attempt {attempt + 1}: Gagal klik tandai, scroll dan retry")
            driver.execute_script("window.scrollBy(0, window.innerHeight / 3);")
            time.sleep(0.5)
            if attempt == max_retry - 1:
                try:
                    tandai_btn = driver.find_element(By.CSS_SELECTOR, "button.btn-tandai")
                    driver.execute_script("arguments[0].click();", tandai_btn)
                    print("Tombol Tandai diklik via JavaScript")
                    tandai_clicked = True
                except:
                    print("Gagal klik tombol Tandai")
                    return False
    
    if not tandai_clicked:
        return False
    
    time.sleep(0.75)
    
    # Isi form keberadaan usaha
    select_gc = driver.find_element(By.ID, "tt_hasil_gc")
    select_gc.click()
    time.sleep(0.3)
    select_gc.send_keys(keberadaan_usaha)
    select_gc.send_keys(Keys.ENTER)
    time.sleep(0.3)
    
    # Isi latitude
    lat_input = driver.find_element(By.ID, "tt_latitude_cek_user")
    lat_input.clear()
    lat_input.send_keys(latitude)
    time.sleep(0.3)
    
    # Isi longitude
    lon_input = driver.find_element(By.ID, "tt_longitude_cek_user")
    lon_input.clear()
    lon_input.send_keys(longitude)
    time.sleep(0.3)
    
    # Klik tombol TANDAI USAHA SUDAH DICEK
    save_btn = driver.find_element(By.ID, "save-tandai-usaha-btn")
    save_btn.click()
    print("Form berhasil disubmit")
    time.sleep(1.5)
    
    # Klik tombol Ya jika muncul
    try:
        ya_btn = driver.find_element(By.CSS_SELECTOR, "button.swal2-confirm")
        if ya_btn.is_displayed() and ya_btn.text.strip() == "Ya":
            ya_btn.click()
            print("Tombol 'Ya' diklik")
            time.sleep(0.75)
    except:
        pass
    
    # Klik tombol OK
    try:
        ok_btn = driver.find_element(By.CSS_SELECTOR, "button.swal2-confirm")
        if ok_btn.is_displayed():
            ok_btn.click()
            print("Tombol 'OK' diklik")
            time.sleep(0.75)
    except:
        pass
    
    return True

## fungsi reverse

In [ ]:
def reverse_gc(df, sumber):
    """
    Fungsi untuk proses GC dengan infinite loop
    Stop dengan Ctrl+C (Keyboard Interrupt)
    """
    processed_count = 0
    no_card_attempt = 0  # Counter untuk tidak ada card
    
    try:
        # ==================== SETUP AWAL ====================
        outlier = []
        # Refresh ke halaman dirgc
        driver.get("https://matchapro.web.bps.go.id/dirgc")
        time.sleep(2.5)
        # Scroll ke home
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(0.75)

        # 1. Pastikan pencarian & filter terexpand
        try:
            filter_input = driver.find_element(By.ID, "search-idsbr")
            if not filter_input.is_displayed():
                driver.find_element(By.ID, "toggle-filter").click()
                time.sleep(1)
        except:
            driver.find_element(By.ID, "toggle-filter").click()
            time.sleep(1)

        # Scroll pgdown
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        time.sleep(0.75)

        # Klik dropdown status GC
        driver.find_element(By.ID, "select2-f_gc-container").click()
        time.sleep(0.75)

        # Pilih "Belum GC"
        driver.find_element(By.CSS_SELECTOR, "li.select2-results__option[data-select2-id*='BELUM']").click()
        time.sleep(3)

        # Klik dropdown sumber data dengan error handling
        try:
            driver.find_element(By.ID, "select2-f_sumber_data-container").click()
        except:
            # Scroll sedikit ke atas
            driver.execute_script("window.scrollBy(0, -200);")
            time.sleep(0.5)
            driver.find_element(By.ID, "select2-f_sumber_data-container").click()
        
        time.sleep(0.75)

        # Pilih opsi yang mengandung teks dari variable sumber
        driver.find_element(By.XPATH, f"//li[contains(@class, 'select2-results__option') and contains(text(), '{sumber}')]").click()
        time.sleep(1)

        # Cek apakah ada pesan "Tidak ada data ditemukan"
        try:
            empty_state = driver.find_element(By.CSS_SELECTOR, "div.empty-state p")
            if empty_state.is_displayed():
                print(empty_state.text)
        except:
            pass

        # Verifikasi filter sudah terpilih dengan benar
        max_verify_attempts = 3
        for verify_attempt in range(max_verify_attempts):
            try:
                # Cek apakah "Belum GC" benar-benar terpilih
                selected_gc = driver.find_element(By.ID, "select2-f_gc-container").text.strip()
                selected_sumber = driver.find_element(By.ID, "select2-f_sumber_data-container").text.strip()
                
                if "BELUM" in selected_gc.upper() and sumber in selected_sumber:
                    print(f"✓ Filter terverifikasi: {selected_gc} | {selected_sumber}")
                    break
                else:
                    raise Exception("Filter belum terpilih dengan benar")
            except:
                print(f"Filter belum siap (attempt {verify_attempt + 1}/{max_verify_attempts}), refresh...")
                driver.get("https://matchapro.web.bps.go.id/dirgc")
                time.sleep(2.5)
                
                # Setup ulang dari awal
                driver.execute_script("window.scrollTo(0, 0);")
                time.sleep(0.75)
                
                try:
                    filter_input = driver.find_element(By.ID, "search-idsbr")
                    if not filter_input.is_displayed():
                        driver.find_element(By.ID, "toggle-filter").click()
                        time.sleep(1)
                except:
                    driver.find_element(By.ID, "toggle-filter").click()
                    time.sleep(1)
                
                driver.execute_script("window.scrollBy(0, window.innerHeight);")
                time.sleep(0.75)
                
                driver.find_element(By.ID, "select2-f_gc-container").click()
                time.sleep(0.75)
                driver.find_element(By.CSS_SELECTOR, "li.select2-results__option[data-select2-id*='BELUM']").click()
                time.sleep(3)
                
                try:
                    driver.find_element(By.ID, "select2-f_sumber_data-container").click()
                except:
                    driver.execute_script("window.scrollBy(0, -200);")
                    time.sleep(0.5)
                    driver.find_element(By.ID, "select2-f_sumber_data-container").click()
                
                time.sleep(0.75)
                driver.find_element(By.XPATH, f"//li[contains(@class, 'select2-results__option') and contains(text(), '{sumber}')]").click()
                time.sleep(1)
                
                if verify_attempt == max_verify_attempts - 1:
                    print("Gagal verifikasi filter setelah 3 attempt, stop")
                    return
        
        # ==================== MULAI LOOP ====================
        while True:
            # Langkah 1: Find badge yang bukan duplikat
            card_headers = driver.find_elements(By.CSS_SELECTOR, "div.usaha-card-header")
            
            found_card = None
            for card_header in card_headers:
                try:
                    status_span = card_header.find_element(By.CSS_SELECTOR, "span.usaha-status")
                    if status_span.text.strip() != "Duplikat":
                        found_card = card_header
                        break
                except:
                    continue
            
            # Langkah 2: If not found, scroll ke bawah 1/2 halaman
            if not found_card:
                no_card_attempt += 1
                print(f"Tidak ada card non-duplikat, scroll ke bawah ({no_card_attempt}/6)")
                
                # Cek apakah sudah 6 kali attempt
                if no_card_attempt >= 6:
                    print("6 attempt tercapai, cooldown 10 detik...")
                    time.sleep(10)
                    no_card_attempt = 0  # Reset counter
                    print("Counter di-reset, coba lagi...")
                
                # UBAH: Langsung scroll ke end dan klik load more
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(0.75)
                
                # Langsung coba klik load more
                try:
                    load_more_btn = driver.find_element(By.ID, "load-more-btn")
                    if load_more_btn.is_displayed():
                        try:
                            load_more_btn.click()
                            print("Klik 'Muat Lebih Banyak'...")
                            time.sleep(2)
                            no_card_attempt = 0  # Reset counter setelah load more
                        except:
                            # Tombol ada tapi belum bisa diklik
                            pass
                except:
                    # Benar-benar sudah tidak ada
                    print("\n=== Tombol Muat Lebih Banyak sudah tidak ada, SBR sudah habis ===")
                    print(f"Total card berhasil di-GC: {processed_count}")
                    break
                
                continue
                
            # Reset counter karena card ditemukan
            no_card_attempt = 0
            
            # Langkah 4: Cek apakah card punya badge "Sudah GC"
            try:
                gc_badge = found_card.find_element(By.CSS_SELECTOR, "span.gc-badge")
                if gc_badge.is_displayed() and "Sudah GC" in gc_badge.text:
                    # Langkah 5: Skip dan ulangi dari awal
                    print("Card sudah GC, skip")
                    driver.execute_script("window.scrollBy(0, window.innerHeight / 2);")
                    time.sleep(0.5)
                    continue
            except:
                # Tidak ada badge "Sudah GC", lanjut proses
                pass
            
            # Langkah 6: Card belum GC, ambil info dan klik untuk expand
            try:
                status = found_card.find_element(By.CSS_SELECTOR, "span.usaha-status").text.strip()
                nama_usaha = found_card.find_element(By.CSS_SELECTOR, "h5.usaha-card-title").text.strip()
                print(f"\n[{processed_count + 1}] Status: {status}")
                print(f"Nama Usaha: {nama_usaha}")
            except:
                print("Gagal ambil info card, skip")
                driver.execute_script("window.scrollBy(0, window.innerHeight / 2);")
                time.sleep(0.5)
                continue
            
            # Klik card untuk expand
            try:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", found_card)
                time.sleep(0.3)
                found_card.click()
                print("Card di-expand")
                time.sleep(0.5)
            except:
                print("Gagal expand card, skip")
                driver.execute_script("window.scrollBy(0, window.innerHeight / 2);")
                time.sleep(0.5)
                continue
            
            # Scroll sedikit ke bawah
            driver.execute_script("window.scrollBy(0, window.innerHeight / 5);")
            time.sleep(0.5)
            
            # Panggil fungsi_tandai
            try:
                hasil = fungsi_tandai(df)
                if hasil:
                    processed_count += 1
                    print(f"✓ Berhasil (Total: {processed_count})")
                    
                    # Time sleep 2 detik
                    time.sleep(2)
                    
                    # Refresh ke halaman dirgc
                    print("Refresh halaman...")
                    driver.get("https://matchapro.web.bps.go.id/dirgc")
                    time.sleep(2.5)
                    
                    # Setup filter lagi
                    driver.execute_script("window.scrollTo(0, 0);")
                    time.sleep(0.75)
                    
                    try:
                        filter_input = driver.find_element(By.ID, "search-idsbr")
                        if not filter_input.is_displayed():
                            driver.find_element(By.ID, "toggle-filter").click()
                            time.sleep(1)
                    except:
                        driver.find_element(By.ID, "toggle-filter").click()
                        time.sleep(1)
                    
                    driver.execute_script("window.scrollBy(0, window.innerHeight);")
                    time.sleep(0.75)
                    
                    driver.find_element(By.ID, "select2-f_gc-container").click()
                    time.sleep(0.75)
                    driver.find_element(By.CSS_SELECTOR, "li.select2-results__option[data-select2-id*='BELUM']").click()
                    time.sleep(3)
                    
                    #sumber = "Dir Pajak"
                    try:
                        driver.find_element(By.ID, "select2-f_sumber_data-container").click()
                    except:
                        driver.execute_script("window.scrollBy(0, -200);")
                        time.sleep(0.5)
                        driver.find_element(By.ID, "select2-f_sumber_data-container").click()
                    
                    time.sleep(0.75)
                    driver.find_element(By.XPATH, f"//li[contains(@class, 'select2-results__option') and contains(text(), '{sumber}')]").click()
                    time.sleep(1)
                    
                    try:
                        empty_state = driver.find_element(By.CSS_SELECTOR, "div.empty-state p")
                        if empty_state.is_displayed():
                            print(empty_state.text)
                    except:
                        pass
                    
                else:
                    print("✗ Gagal")
                    time.sleep(2)
                    # Klik card untuk collapse
                    try:
                        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", found_card)
                        time.sleep(0.3)
                        found_card.click()
                        print("Card di-collapse")
                        time.sleep(0.5)
                    except:
                        print("Gagal collapse card")
                    
            except Exception as e:
                print(f"✗ Error: {str(e)}")
                time.sleep(2)
                # Klik card untuk collapse
                try:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", found_card)
                    time.sleep(0.3)
                    found_card.click()
                    print("Card di-collapse")
                    time.sleep(0.5)
                except:
                    print("Gagal collapse card")
            
    except KeyboardInterrupt:
        print(f"\n\n=== Proses dihentikan ===")
        print(f"Total card berhasil di-GC: {processed_count}")


# EKSEKUSI

In [132]:
# Jalankan proses infinite scroll untuk tandai semua card non-duplikat
sumber = "All"
# PL-KUMKM 2023/OSS - Perorangan/OSS - Badan Usaha
# Dir Pajak/Tambahan Daerah s.d 31 Desember 2025
reverse_gc(sbrclean, sumber)

Tidak ada card non-duplikat, scroll ke bawah (1/6)
Tidak ada card non-duplikat, scroll ke bawah (2/6)
Klik 'Muat Lebih Banyak'...
Tidak ada card non-duplikat, scroll ke bawah (1/6)
Tidak ada card non-duplikat, scroll ke bawah (2/6)
Klik 'Muat Lebih Banyak'...
Tidak ada card non-duplikat, scroll ke bawah (1/6)
Tidak ada card non-duplikat, scroll ke bawah (2/6)
Klik 'Muat Lebih Banyak'...
Tidak ada card non-duplikat, scroll ke bawah (1/6)
Tidak ada card non-duplikat, scroll ke bawah (2/6)
Klik 'Muat Lebih Banyak'...
Tidak ada card non-duplikat, scroll ke bawah (1/6)
Tidak ada card non-duplikat, scroll ke bawah (2/6)
Klik 'Muat Lebih Banyak'...
Tidak ada card non-duplikat, scroll ke bawah (1/6)
Tidak ada card non-duplikat, scroll ke bawah (2/6)
Klik 'Muat Lebih Banyak'...
Tidak ada card non-duplikat, scroll ke bawah (1/6)
Tidak ada card non-duplikat, scroll ke bawah (2/6)
Klik 'Muat Lebih Banyak'...

[1] Status: Tidak Ditemukan
Nama Usaha: PUTRA MANDIRI PT
Card di-expand
IDSBR: 97318477
D

# debug doang ini, ga penting

In [130]:
# Cek dan klik tombol "Muat Lebih Banyak"
try:
    load_more_btn = driver.find_element(By.ID, "load-more-btn")
    load_more_btn.click()
    print("Tombol 'Muat Lebih Banyak' berhasil diklik")
    time.sleep(2)
except Exception as e:
    print(f"Gagal klik tombol 'Muat Lebih Banyak': {e}")


Gagal klik tombol 'Muat Lebih Banyak': Message: element click intercepted: Element is not clickable at point (249, 1138)
  (Session info: chrome=144.0.7559.110)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6deb3a535
	0x7ff6deb3a590
	0x7ff6de8e10bd
	0x7ff6de944a15
	0x7ff6de94212a
	0x7ff6de93efb5
	0x7ff6de93de50
	0x7ff6de92f076
	0x7ff6de965a3a
	0x7ff6de92e8f6
	0x7ff6de989ef5
	0x7ff6de92cc9c
	0x7ff6de92dbe3
	0x7ff6dee17b60
	0x7ff6dee11f5d
	0x7ff6dee3311a
	0x7ff6deb560a5
	0x7ff6deb5e73c
	0x7ff6deb43844
	0x7ff6deb439f6
	0x7ff6deb29a07
	0x7ffc1242e8d7
	0x7ffc13aac53c



# scroll ke home

In [124]:
driver.execute_script("window.scrollTo(0, 0);")
time.sleep(1)

# ini buat expand/collapse card usaha
pastikan semua card state nya sama (semua ter-expand atau semua ter-collapse)

In [125]:
# Klik semua card dengan status "Duplikat"
duplikat_cards = driver.find_elements(By.CSS_SELECTOR, "span.usaha-status.tidak-aktif")

print(f"Total card duplikat ditemukan: {len(duplikat_cards)}")

for i, card in enumerate(duplikat_cards, 1):
    try:
        # Scroll ke elemen
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", card)
        time.sleep(0.3)
        
        # Klik card (klik parent header-nya)
        card_header = card.find_element(By.XPATH, "./ancestor::div[@class='usaha-card-header']")
        card_header.click()
        print(f"[{i}/{len(duplikat_cards)}] Card duplikat diklik")
        time.sleep(0.2)
    except Exception as e:
        print(f"[{i}/{len(duplikat_cards)}] Gagal klik: {str(e)}")
        continue

print("Selesai mengklik semua card duplikat")


Total card duplikat ditemukan: 17
[1/17] Card duplikat diklik
[2/17] Card duplikat diklik
[3/17] Card duplikat diklik
[4/17] Card duplikat diklik
[5/17] Card duplikat diklik
[6/17] Card duplikat diklik
[7/17] Card duplikat diklik


KeyboardInterrupt: 

# ini buat Get all IDSBR dari card yang terexpand

In [120]:
list_idsbr = []

# Cari semua card yang expanded
expanded_cards = driver.find_elements(By.CSS_SELECTOR, "div.usaha-card.expanded")

for card in expanded_cards:
    try:
        # Cari detail rows di dalam card ini
        detail_rows = card.find_elements(By.CSS_SELECTOR, "div.detail-row")
        
        for row in detail_rows:
            label = row.find_element(By.CSS_SELECTOR, "span.detail-label").text.strip()
            if label == "#IDSBR":
                idsbr = row.find_element(By.CSS_SELECTOR, "span.detail-value").text.strip()
                list_idsbr.append(idsbr)
                break
    except:
        continue

print(f"Total IDSBR ditemukan: {len(list_idsbr)}")
print(list_idsbr)


Total IDSBR ditemukan: 134
['2635350', '2686899', '2607304', '2707232', '2681803', '2568039', '2570129', '2517118', '2712168', '2665011', '2862872', '2854418', '2804386', '2866747', '2790023', '2776522', '2820492', '2737492', '2765222', '2869564', '2958173', '2963113', '2946926', '2928261', '2942042', '2968928', '2945323', '2964260', '2915852', '2960662', '3055414', '3057978', '3025639', '2979467', '3016930', '3005284', '2973957', '2985462', '3045926', '3039613', '3361083', '3307877', '3321839', '3189959', '3173600', '3153948', '3148021', '3059810', '3288633', '3153643', '3361581', '3363842', '3368161', '3371319', '3430915', '3455687', '3534908', '3543867', '3555251', '3565042', '3642635', '3606623', '3608425', '3665093', '3639203', '3657667', '3659432', '3574442', '3634424', '3632160', '3796665', '3807975', '3813645', '3669915', '3694630', '3727499', '3758539', '3789787', '3807177', '3747076', '3827524', '3849805', '3839525', '3827981', '3864613', '3816349', '3847623', '3814302', '388

In [121]:
# Import file lokal yang sudah ada
try:
    df_existing = pd.read_excel(r'C:\Users\Acer\Downloads\list_idsbr.xlsx')
    print(f"File existing ditemukan, total IDSBR: {len(df_existing)}")
except FileNotFoundError:
    # Kalau file belum ada, buat DataFrame kosong
    df_existing = pd.DataFrame(columns=['IDSBR'])
    print("File tidak ditemukan, buat baru")

# Buat DataFrame dari list_idsbr baru
df_new = pd.DataFrame(list_idsbr, columns=['IDSBR'])

# Gabungkan (append) dengan data existing
df_combined = pd.concat([df_existing, df_new], ignore_index=True)

# Hapus duplikat (opsional)
df_combined = df_combined.drop_duplicates(subset=['IDSBR'], keep='first')

# Export ke file lokal
df_combined.to_excel(r'C:\Users\Acer\Downloads\list_idsbr.xlsx', index=False)

print(f"Total IDSBR setelah digabung: {len(df_combined)}")
print(f"IDSBR baru yang ditambahkan: {len(df_new)}")

File existing ditemukan, total IDSBR: 17
Total IDSBR setelah digabung: 151
IDSBR baru yang ditambahkan: 134
